In [ ]:
# Connect to Google Drive!
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Change working directory

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import cv2
import torch
from PIL import Image
import pandas as pd
import sys
import numpy as np

In [ ]:
file_name = "2403_g6_sync_yolov5x6_resized_corrected"

In [ ]:
model = torch.hub.load("ultralytics/yolov5", "yolov5x6", force_reload=False) # Can change weights eg: yolov5s

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 606 layers, 141755500 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 2021-5-16 torch 1.8.1+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)



In [ ]:
cap= cv2.VideoCapture("/content/drive/MyDrive/Bachelor Project/Videos/24032021/Processed/2403_G6_sync.mp4")

# Resized G6
K = np.array([[875.53696495, 0, 962.77164738],
              [0, 875.39040838, 538.76818232],
              [0, 0, 1,]])

d = np.array([-0.04141805, -0.03777076, 0.00092132, -0.00010891, 0.01265788])

# Resized S7
#K = np.array([[1.50812622e+03, 0.00000000e+00, 9.89113794e+02],
#              [0.00000000e+00, 1.50622318e+03, 5.03874158e+02],
#              [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

#d = np.array([4.49989117e-01, -1.51139444e+00, 1.62046172e-03, 3.92046381e-03, 1.72520519e+00])

h, w = 1080, 1920

newcameramatrix, roi = cv2.getOptimalNewCameraMatrix(K, d, (w,h), 1)

mapx, mapy = cv2.initUndistortRectifyMap(K, d, None, newcameramatrix, (w, h), 5)

In [ ]:
model.classes = [1]

df = pd.DataFrame(columns=["xmin", "ymin", "xmax", "ymax", "confidence", "frame_id"])

count=0
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

while(cap.isOpened()):
    sys.stdout.write("\r" + f"{count} frame of {length} - {round((count/length)*100, 2)} %")
    sys.stdout.flush()
    ret, frame = cap.read()
    if ret == False:
        break
    newimg = cv2.remap(frame, mapx, mapy, cv2.INTER_LINEAR)
    results = model(newimg, size=1280) 
    count+=1
    data = {"xmin": results.pandas().xyxy[0]["xmin"], "ymin": results.pandas().xyxy[0]["ymin"],
            "xmax": results.pandas().xyxy[0]["xmax"], "ymax": results.pandas().xyxy[0]["ymax"],
            "confidence": results.pandas().xyxy[0]["confidence"], "frame_id": count}

    temp = pd.DataFrame(data=data)
    df = pd.concat([df, temp])

cap.release()
cv2.destroyAllWindows()

165330 frame of 165440 - 99.93 %

In [ ]:
df.to_pickle(f"/content/drive/MyDrive/{file_name}.pickle")

In [ ]:
img = cv2.imread("/content/2403_G6_sync.jpg")
newimg = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
cv2.imwrite("/content/drive/MyDrive/straight_inter.jpg", newimg)

True